In [27]:
!pip install seaborn

In [1]:
import sys
sys.path.append('..')

In [128]:
from contexto.guess import Finder
import numpy as np
import seaborn as sns
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt


In [130]:
embedding = TSNE(n_components=2, learning_rate='auto', metric="cosine").fit_transform(Finder().embeddings)

In [281]:
def plot(scores, save_id=None):
    is_selected = scores > 0

    plt.figure(figsize=(15,8))

    not_selected_embedding = embedding[~is_selected]
    selected_embedding = embedding[is_selected]

    top_score = max(scores)

    top_embedding = embedding[scores == top_score][:1]


    ax = sns.scatterplot(
        x=not_selected_embedding[:, 0],
        y=not_selected_embedding[:, 1],
        legend="",
        size=1,
        hue=0.1,
    )

    ax = sns.scatterplot(
        x=selected_embedding[:, 0],
        y=selected_embedding[:, 1],
        size=scores[is_selected],
        # hue=scores[is_selected],
        hue=0,
        legend="",
        ax=ax
    )

    ax = sns.scatterplot(
        x=top_embedding[:, 0],
        y=top_embedding[:, 1],
        ax=ax,
    )

    ax = sns.scatterplot(
        x=top_embedding[:, 0],
        y=top_embedding[:, 1],
        ax=ax,
    )

    plt.tight_layout()


    # remove axis
    ax.set_xticks([])
    ax.set_yticks([])

    if save_id is not None:
        plt.savefig(f'../plots/{save_id:02d}.png')
    else:
        plt.show()

In [275]:
def score(guesses):
    finder = Finder()
    for guess, rank in guesses:
        finder.add_result(guess, rank)
    scores = finder.get_score(
        guesses=guesses,
        word_to_distances=finder.word_to_distances,
        min_gap=0.2,
        num_samples=1000
    )
    already_guessed_masked = finder.already_guessed_mask(guesses)
    scores[already_guessed_masked] = 0
    best_scores_masked = finder.best_scores(guesses=guesses, word_to_distances=finder.word_to_distances, top=100)
    scores[~best_scores_masked] = 0

    top_score = max(scores)
    closest_ids = list(np.arange(len(finder.words))[scores == top_score])
    return scores, [finder.words[idx] for idx in closest_ids]


In [276]:
def process(guesses):
    scores, suggestion = score(guesses)
    print("Guesses: ", suggestion[:5])
    plot(scores)


In [ ]:
process([
    ('human', ...),
])